# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from numpy.linalg import inv
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('insurance.csv')

In [3]:
display(data.head(5))

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

In [5]:
display(data.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
display(data.duplicated().sum())

153

В нашем распоряжении датасет, состоящий из 5 столбцов и 5000 строк. Пропущенных значений нет. Есть дубликаты, но мы их оставим, так как они не повлияют на решение нашей задачи.

Пол, возраст, зарплата и количество членов семьи - features.
Страховые выплаты - target.

Данные готовы.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

***Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?***

**Ответ:**  При умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

**Обоснование:** ...

Домножим наши признаки на некоторую случайную матрицу A, при условии что число её строк равно числу столбцов X и А — обратима, то есть существует такая $ A^{-1} $ что $ A^{-1}A = E $, где E - единичная матрица.

Пусть $Z=XA$. Посмотрим чему будет равно предсказание и вектор весов:

$
a_{1} = Zw_{-1}
$

$
w_{1} = (Z^T Z)^{-1} Z^T y
$

Объединим эти формулы:

$
a_{1} = Z(Z^T Z)^{-1} Z^T y
$

Заменим Z на XA:

$
a_{1} = XA((XA)^T (XA))^{-1} (XA)^T y
$

Используя свойство обратной матрицы $(AB)^{-1}=B^{-1}A^{-1}$ преобразуем предыдущую формулу:

$
a_{1} = XA((XA)^T (XA))^{-1} (XA)^T y = XA (XA)^{-1} ((XA)^T)^{-1} (XA)^T y = XA A^{-1} X^{-1}((XA)^T)^{-1} (XA)^T y
$

Так как $ A^{-1}A = E $ и $(AB)^T = B^T A^T $ (свойство транспонированных матриц) получаем:

$
a_{1} = XA A^{-1} X^{-1}((XA)^T)^{-1} (XA)^T y = XEX^{-1}(A^T X^T)^{-1} A^T X^T y = XX^{-1}(A^T)^{-1}(X^T)^{-1}A^T X^T y
$

Так как $(A^T)^{-1} A^T = E$ получаем:

$
a_{1} = XX^{-1}(A^T)^{-1}(X^T)^{-1}A^T X^T y = XX^{-1} E (X^T)^{-1} X^T y = X (X^T X)^{-1}X^T y = Xw = a
$

Таким образом получается $a=a_{1}$, а это значит, что значения предсказаний не изменятся, если умножить матрицу признаков на обратимую матрицу.



$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$

$
w' = ((XP)^T XP)^{-1} (XP)^T y
$

$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$

$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$

$
w' = P^{-1} (X^T X)^{-1} E X^T y
$

$
w' = P^{-1} (X^T X)^{-1} X^T y
$

$
w' = P^{-1} w
$


## Алгоритм преобразования

**Алгоритм**

Умножение матрицы признаков на обратимую матрицу, которая будет генерироваться случайным образом.

**Обоснование**

Создадим случайную матрицу размера 4*4:

In [7]:
random_matrix = np.random.rand(4,4)
display(random_matrix)

array([[0.03946364, 0.19233663, 0.17559752, 0.65210237],
       [0.59659526, 0.29340839, 0.12990802, 0.35476262],
       [0.02753429, 0.11676805, 0.73118293, 0.65913172],
       [0.59092501, 0.84647168, 0.0856697 , 0.06108777]])

Создадим обратную ей матрицу (чтобы точно знать, что матрица для нашего алгоритма будет обратимой)

In [8]:
inv_matrix = np.linalg.inv(random_matrix)
display(inv_matrix)

array([[-1.1112024 ,  2.27323125, -0.07550134, -0.52505483],
       [ 0.81956908, -1.59685006, -0.09611786,  1.56191822],
       [-1.73539492, -0.17319584,  1.79089355,  0.2073052 ],
       [ 1.82632363,  0.3800562 , -0.44933117, -0.48473339]])

Наша матрица для умножения готова, можно проверять алгоритм.

## Проверка алгоритма

Разделим данные на фичи и таргет, а потом на трейн и тест:

In [9]:
features = data.drop('Страховые выплаты',axis=1)
target = data['Страховые выплаты']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=111)

Обучим модель и посчитаем R2:

In [10]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(r2_score(target_test, predictions))

0.4583166347384783


Теперь умножим матрицы фичей на обратимую матрицу:

In [11]:
new_features_train = features_train@inv_matrix
new_features_test = features_test@inv_matrix

Снова обучим модель и посчитаем R2:

In [12]:
model = LinearRegression()
model.fit(new_features_train, target_train)
predictions = model.predict(new_features_test)
print(r2_score(target_test, predictions))

0.45831663473855255


Вывод: Качество линейной регрессии (метрика R2) не отличается до и после преобразоваия. 